TO DO:
[] interpret coeff & feature importance

[] model without pca (weak correlations)


# 04. Modeling
___


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split


from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import ConfusionMatrixDisplay, classification_report
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, roc_curve, roc_auc_score

from sklearn.model_selection import GridSearchCV




___


Introduction


In this notebook, I will be building two types classification models. Using Logistic Regression and Decision Trees, we will see if we can correctly classify our target variable and predict the classes of new datapoints.
I will train each model with a portion of the dataset and then compare the outputs with my test set. 


___


In [2]:
heart22 = pd.read_csv('~/Desktop/capstone-project-Tasnimacj/data/cleaned_data/heart22_preprocessed.csv')

In [3]:
heart22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246013 entries, 0 to 246012
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     246013 non-null  int64  
 1   Female                         246013 non-null  int64  
 2   GeneralHealth                  246013 non-null  int64  
 3   PhysicalHealthDays             246013 non-null  float64
 4   MentalHealthDays               246013 non-null  float64
 5   LastCheckupTime                246013 non-null  int64  
 6   PhysicalActivities             246013 non-null  int64  
 7   SleepHours                     246013 non-null  float64
 8   RemovedTeeth                   246013 non-null  int64  
 9   HadHeartAttack                 246013 non-null  int64  
 10  HadAngina                      246013 non-null  int64  
 11  HadStroke                      246013 non-null  int64  
 12  HadAsthma                     

Here I load in the data that we saved from the previous notebook. Checking if our dataset is properly encoded, with 0 null values and has a correct index. Everything looks good, so we can move straight into modeling now.

In [5]:
y = heart22['HadAngina'] # Target Variable
X = heart22.drop('HadAngina', axis=1) 

In [6]:
print('Shape of y:', y.shape)
print('Shape of X:', X.shape)

Shape of y: (246013,)
Shape of X: (246013, 42)


Split our data into our feature columns and our target variable. We will be putting our X into our models and comparing the outputs with our y.


In [7]:
#1st split

X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size=0.2, random_state=25, stratify=y)

print(f'The remainder set has {len(X_rem)} data points.')
print(f'The test set has {len(X_test)} data points.')

The remainder set has 196810 data points.
The test set has 49203 data points.


For the model, we need to split our data into a training set and a test set. Here I did a 80:20 split, keeping an even distribution of y variables in each split. 
We use the train set to fit the model and then evaluate on the test set. Splitting the datapoints helps prevent overfitting and a way to accurately check model performance. By having unseen data, we can come close to replicating real life scenarios that the model will face.

___

### 1 Baseline Logistic Regression


For my first model, I will attempt to create a model that has no hyperparameter optimization. This will be a a model we can use for comparison. I am expecting that this model will perform poorly and will be very good at predicting 0, or 'did not have angina'
As logistic regression takes L2 Regularisation as its default penalty, I will need to scale my data beforehand.

In [67]:
ss = StandardScaler().fit(X_rem)
X_rem_ss = ss.transform(X_rem)
X_test_ss = ss.transform(X_test)

In [64]:
baseline_log_reg = LogisticRegression(random_state=1)
baseline_log_reg.fit(X_rem_ss, y_rem)

LogisticRegression(random_state=1)

We instantiate the model and then fit the model on our y remainder data and the scaled X remainder data. We then input unknown data and compare accuracy scores with our test data.

In [65]:
print(f'Accuracy on remainder set: {baseline_log_reg.score(X_rem_ss, y_rem)}')
print(f'Accuracy on test set: {baseline_log_reg.score(X_test_ss, y_test)}')

Accuracy on remainder set: 0.9450586860423759
Accuracy on test set: 0.9446375221023109


Our accuracy on the remainder and test set is the same, 94.5%. This means our model has correctly classified 94.5% of the data points in both sets. This shows that the model has a good fit, and can handle unseen data.

However, accuracy is not a good indicator of the model outcomes as it does not show if our model is behaving how we expect. 
We can look further by using a classification report and confusion matrix to see what datapoints were classified as.

In [72]:

y_test_pred = baseline_log_reg.predict(X_test_ss)


cmat = pd.DataFrame(
    data = confusion_matrix(y_test, y_test_pred),
    index = ['true 0', 'true 1'],
    columns = ['predicted 0', 'predicted 1']
)
display(cmat)


print(f'Recall score: {recall_score(y_test, y_test_pred)*100:0.2f}%')
print(f'Precision score: {precision_score(y_test, y_test_pred)*100:0.2f}%')
print(f'F1 score: {f1_score(y_test, y_test_pred)*100:0.2f}%')


,predicted 0,predicted 1
true 0,45642,570
true 1,2154,837


Recall score: 27.98%
Precision score: 59.49%
F1 score: 38.06%


In [73]:

print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97     46212
           1       0.59      0.28      0.38      2991

    accuracy                           0.94     49203
   macro avg       0.77      0.63      0.68     49203
weighted avg       0.93      0.94      0.94     49203



Even though our model was 94.5% accurate, it was only good at predicting 'Had no Angina'. This is most likely due to the imbalance in our dataset. It has seen more cases where y would be '0' than when y would be '1'.

 My main goal is to get high recall without sacrificing precision and accuracy. For my problem, it would be better to have more false positives than false negatives. It would be worse to mischaracterise someone as not at risk than saying they could be at risk. My model could only classify 837 datapoints correcty as '1' out of 2991 '1' values.

___

### 2 Tuning Logistic Regression

Pipeline

Using a pipeline, I want to find the best hyperparameters for my model. I set up a pipeline that contains a scaler, dimension reducer and a model of our choice. 

In [13]:
from tempfile import mkdtemp
cachedir = mkdtemp()

In [14]:
pipe = Pipeline([("scaler", StandardScaler()),
                 ("my_pca", PCA(n_components=20)),
                 ("model", LogisticRegression())], memory=cachedir)

Hyperparameter Optimisation

Setting up a parameter grid with what I would like to change in my model. I want to explore different c values and the model penalty. For now, I only want to scale my data using a standard scaler and look at the top 20 components.

In [18]:

c_values = [.0001, .001, .01, .1, 1, 10, 100, 1000, 10000]


log_reg_param = [

    {'scaler': [ StandardScaler()],
     'my_pca__n_components': [20],
     'model': [LogisticRegression(solver='saga',random_state=1, n_jobs=-1, max_iter=10000)], 
     'model__C': c_values,
     'model__penalty': ['l1', 'l2'],
    }
]

GridSearch

In [19]:
grid = GridSearchCV(estimator=pipe,param_grid=log_reg_param, cv=5,verbose=1,refit=True)

In [20]:
fittedgrid_lr = grid.fit(X_rem,y_rem)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


In [21]:
fittedgrid_lr.best_params_

{'model': LogisticRegression(C=0.001, max_iter=10000, n_jobs=-1, random_state=1,
                    solver='saga'),
 'model__C': 0.001,
 'model__penalty': 'l2',
 'my_pca__n_components': 20,
 'scaler': StandardScaler()}

GridSearch looks at different parameter settings across 5 cross folds on our Remainder set. It does this to find the best model settings whilst preventing data leakage and overfitting on our train set. 
GridSearch has found that the best parameters for our Logistic Regression is having a C value of 0.001, which means we need a strong regularization strength. We need a l2 penalty to help shrink coefficients to 0.

In [23]:
print(f"Best accuracy on the remainder set: {fittedgrid_lr.score(X_rem, y_rem)}")
print(f"Best accuracy on the test set: {fittedgrid_lr.score(X_test, y_test)}")

Best accuracy on the remainder set: 0.9424876784716224
Best accuracy on the test set: 0.9429709570554641


In [24]:
y_test_pred = fittedgrid_lr.predict(X_test)

conmat = pd.DataFrame(
    data = confusion_matrix(y_test, y_test_pred),
    index = ['true 0', 'true 1'],
    columns = ['predicted 0', 'predicted 1']
)
display(conmat)

print(f'Recall score: {recall_score(y_test, y_test_pred)*100:0.2f}%')
print(f'Precision score: {precision_score(y_test, y_test_pred)*100:0.2f}%')
print(f'F1 score: {f1_score(y_test, y_test_pred)*100:0.2f}%')

,predicted 0,predicted 1
true 0,45845,367
true 1,2439,552


Recall score: 18.46%
Precision score: 60.07%
F1 score: 28.24%


Our decided best logistic regression model is performing worse than our baseline in terms of recall.
 Perhaps PCA downgraded model performance, we can investigate this further in future notebooks.

___

### 3 Tuning Decision Tree

Pipeline

Setting up the pipeline for another classification model, this time I will use Decision Tree Classifier.

In [32]:
pipe = Pipeline([("scaler", StandardScaler()),
                 ("my_pca", PCA(n_components=20)),
                 ("dt_model", DecisionTreeClassifier())], memory=cachedir)

Hyperparameter Optimisation

For the parameter grid, this time I want to see if scaling data would make a change in modeling. I still 20 components for PCA but explore different options for the trees max_depth and min_samples_leaf.

In [36]:

dt_param  = {"scaler":[StandardScaler(), None],
            "my_pca__n_components":[20],
            "dt_model__max_depth": [None, 2, 4, 6, 8,10],
            "dt_model__min_samples_leaf": [2, 5, 10] }

In [31]:
# pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scaler', 'my_pca', 'dt_model', 'scaler__copy', 'scaler__with_mean', 'scaler__with_std', 'my_pca__copy', 'my_pca__iterated_power', 'my_pca__n_components', 'my_pca__random_state', 'my_pca__svd_solver', 'my_pca__tol', 'my_pca__whiten', 'dt_model__ccp_alpha', 'dt_model__class_weight', 'dt_model__criterion', 'dt_model__max_depth', 'dt_model__max_features', 'dt_model__max_leaf_nodes', 'dt_model__min_impurity_decrease', 'dt_model__min_impurity_split', 'dt_model__min_samples_leaf', 'dt_model__min_samples_split', 'dt_model__min_weight_fraction_leaf', 'dt_model__random_state', 'dt_model__splitter'])

GridSearch

In [37]:
grid = GridSearchCV(estimator=pipe,param_grid=dt_param, cv=5,verbose=1,refit=True)

In [38]:
fittedgrid_dt = grid.fit(X_rem,y_rem)


Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [39]:
fittedgrid_dt.best_params_

{'dt_model__max_depth': 6,
 'dt_model__min_samples_leaf': 10,
 'my_pca__n_components': 20,
 'scaler': StandardScaler()}

The best optimization for our decision tree would be using a standard scaler, having a max_depth of 6 and 10 min_samples_leaf. Scaling my data beforehand seems to be a crucial step for modeling.

In [41]:
print(f"Best accuracy on the remainder set: {fittedgrid_dt.score(X_rem, y_rem)}")
print(f"Best accuracy on the test set: {fittedgrid_dt.score(X_test, y_test)}")

Best accuracy on the remainder set: 0.9402977490981149
Best accuracy on the test set: 0.9395768550698128


Our accuracies are, again, similar to each other. We have no issue with over/underfitting. The accuracies being in the 90s shows that it is very confident in predicting classes correctly. 

In [42]:

y_test_pred = fittedgrid_dt.predict(X_test)

conmat = pd.DataFrame(
    data = confusion_matrix(y_test, y_test_pred),
    index = ['true 0', 'true 1'],
    columns = ['predicted 0', 'predicted 1']
)
display(conmat)

print(f'Recall score: {recall_score(y_test, y_test_pred)*100:0.2f}%')
print(f'Precision score: {precision_score(y_test, y_test_pred)*100:0.2f}%')
print(f'F1 score: {f1_score(y_test, y_test_pred)*100:0.2f}%')

,predicted 0,predicted 1
true 0,45967,245
true 1,2728,263


Recall score: 8.79%
Precision score: 51.77%
F1 score: 15.03%


Our recall score has plummeted to 8%. This model is very bad classifying '1's, so it is unusable.
The precision is around 50%, so only about half of its '1' predictions were correct.   

___

### 4 SMOTE

As we saw in EDA, the actual proportion of 'HadAngina' in our target column was very small. To try and combat this we can sample our data, either using Upsampling, Downsampling or SMOTE. Here, I will be using SMOTE to create 'fake' data points by interpolating between datapoints. I hope to see model performances increase as they can learn better from having a more balanced dataset. 

In [45]:
from imblearn.over_sampling import SMOTE

In [46]:
X_rem_sm, y_rem_sm = SMOTE(random_state=1).fit_resample(X_rem, y_rem)

In [79]:

print('Original class distribution')
print((y_rem).value_counts().sort_index(),'\n')

print('Resampled class distribution')
print((y_rem_sm).value_counts().sort_index())

Original class distribution
HadAngina
0    184848
1     11962
Name: count, dtype: int64 

Resampled class distribution
HadAngina
0    184848
1    184848
Name: count, dtype: int64


SMOTE has inflated our '1' distribution to now be equal with the number of '0's.

#### 4.1 LogReg

Repeating the previous steps again but this time fitting with our new balanced remainder set.

In [48]:
pipe = Pipeline([("scaler", StandardScaler()),
                 ("my_pca", PCA(n_components=20)),
                 ("model", LogisticRegression())], memory=cachedir)
log_reg_param = [

    {'scaler': [ StandardScaler()],
     'my_pca__n_components': [20],
     'model': [LogisticRegression(solver='saga',random_state=1, n_jobs=-1, max_iter=10000)], 
     'model__C': c_values,
     'model__penalty': ['l1', 'l2'],
    }
]

grid_sm = GridSearchCV(estimator=pipe,param_grid=log_reg_param, cv=5,verbose=1,refit=True)

In [49]:
fittedgrid_lr_sm = grid_sm.fit(X_rem_sm,y_rem_sm)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


In [59]:

print(f"Best accuracy on the remainder set: {fittedgrid_lr_sm.score(X_rem_sm, y_rem_sm)}")
print(f"Best accuracy on the test set: {fittedgrid_lr_sm.score(X_test, y_test)}")


y_test_pred = fittedgrid_lr_sm.predict(X_test)

conmat = pd.DataFrame(
    data = confusion_matrix(y_test, y_test_pred),
    index = ['true 0', 'true 1'],
    columns = ['predicted 0', 'predicted 1']
)
display(conmat)

print(f'Recall score: {recall_score(y_test, y_test_pred)*100:0.2f}%')
print(f'Precision score: {precision_score(y_test, y_test_pred)*100:0.2f}%')
print(f'F1 score: {f1_score(y_test, y_test_pred)*100:0.2f}%')

Best accuracy on the remainder set: 0.8598605340604172
Best accuracy on the test set: 0.8297868016177875


,predicted 0,predicted 1
true 0,39380,6832
true 1,1543,1448


Recall score: 48.41%
Precision score: 17.49%
F1 score: 25.69%


Although the accuracy has dropped, I can see an increase in our recall. The model became more familiar with cases that are classed as '1'  and can recognise it. However, our precision has fallen quite a bit, which shows that the model is not  predicting correctly. I can further investigate and make more changes to find a model with a better recall score.

#### 4.2 DT

Fitting a decision tree with our balanced data.

In [56]:
pipe = Pipeline([("scaler", StandardScaler()),
                 ("my_pca", PCA(n_components=20)),
                 ("dt_model", DecisionTreeClassifier())], memory=cachedir)

dt_param  = {"scaler":[StandardScaler(), None],
            "my_pca__n_components":[20],
            "dt_model__max_depth": [None, 2, 4, 6, 8,10],
            "dt_model__min_samples_leaf": [2, 5, 10] }


grid_sm = GridSearchCV(estimator=pipe,param_grid=dt_param, cv=5,verbose=1,refit=True)

In [57]:
fittedgrid_dt_sm = grid_sm.fit(X_rem_sm,y_rem_sm)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [60]:
print(f"Best accuracy on the remainder set: {fittedgrid_dt_sm.score(X_rem_sm, y_rem_sm)}")
print(f"Best accuracy on the test set: {fittedgrid_dt_sm.score(X_test, y_test)}")


y_test_pred = fittedgrid_dt_sm.predict(X_test)


conmat = pd.DataFrame(
    data = confusion_matrix(y_test, y_test_pred),
    index = ['true 0', 'true 1'],
    columns = ['predicted 0', 'predicted 1']
)
display(conmat)

print(f'Recall score: {recall_score(y_test, y_test_pred)*100:0.2f}%')
print(f'Precision score: {precision_score(y_test, y_test_pred)*100:0.2f}%')
print(f'F1 score: {f1_score(y_test, y_test_pred)*100:0.2f}%')

Best accuracy on the remainder set: 0.863314723448455
Best accuracy on the test set: 0.8128975875454749


,predicted 0,predicted 1
true 0,38756,7456
true 1,1750,1241


Recall score: 41.49%
Precision score: 14.27%
F1 score: 21.24%


This is a huge increase in our recall score. So we can see that SMOTE does help with prediction.

 But our precision score has dropped again. 

___


Conclusion

In [74]:
data = {'F1 score' :[38.06,28.24,15.03,25.69,21.24],
     'Recall score':[27.98,18.46,8.79,48.41,41.49], 
     'Precision score':[59.49,60.07,51.77,17.49,14.27],
      'Accuracy':[94.46,94.30,93.96,82.98,81.29]}


scores = pd.DataFrame(
    data = data,
    index = ['Baseline LogReg', 'Best LogReg', 'Best DT','Best SMOTE LogReg', 'Best SMOTE DT' ],
    columns = ['F1 score','Recall score', 'Precision score', 'Accuracy']
)

Here is a table that compares the F1, Recall, Precision and Test Accuracy for each model.

In [75]:
scores

,F1 score,Recall score,Precision score,Accuracy
Baseline LogReg,38.06,27.98,59.49,94.46
Best LogReg,28.24,18.46,60.07,94.30
Best DT,15.03,8.79,51.77,93.96
Best SMOTE LogReg,25.69,48.41,17.49,82.98
Best SMOTE DT,21.24,41.49,14.27,81.29


Before sampling, my best model was my baseline model. After sampling, I found that my recall improves, which is what I ultimately want.  In future notebooks, I will be looking at modelling without PCA, and trying different hyperparameters. 
I will have to look into different combinations for my model that would give me a good recall score but also not affect the precision as much.